In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
import theano as th
import theano.tensor as T
import matlab.engine

In [ ]:
# Generate data
X, y = make_classification(n_samples=1000, n_features=20, n_classes=2, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Data preprocessing
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
# Define the SVM model
class SVM(nn.Module):
    def __init__(self):
        super(SVM, self).__init__()
        self.linear = nn.Linear(20, 1)

    def forward(self, x):
        return self.linear(x)

# model and define loss function and optimizer
model = SVM()
criterion = nn.HingeEmbeddingLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Train the model
num_epochs = 100
for epoch in range(num_epochs):
    optimizer.zero_grad()
    outputs = model(torch.tensor(X_train, dtype=torch.float32))
    loss = criterion(outputs.squeeze(), torch.tensor(y_train, dtype=torch.float32))
    loss.backward()
    optimizer.step()

    if (epoch+1) % 10 == 0:
        print(f'PyTorch: Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

In [ ]:
with torch.no_grad():
    outputs = model(torch.tensor(X_test, dtype=torch.float32))
    predicted = torch.sign(outputs).detach().numpy()
    accuracy = (predicted.squeeze() == y_test).mean()
    print(f'PyTorch: Test Accuracy: {accuracy:.4f}')

In [ ]:
class SVMTheano:
    def __init__(self):
        self.w = th.shared(np.random.randn(20), name='w')
        self.b = th.shared(0., name='b')
        self.params = [self.w, self.b]
        self.X = T.matrix('X')
        self.y = T.vector('y')

In [ ]:
def train(self, X_train, y_train, lr=0.01, epochs=100):
        y_pred = T.dot(self.X, self.w) + self.b
        hinge_loss = T.mean(T.maximum(0., 1 - self.y * y_pred))
        gw, gb = T.grad(hinge_loss, [self.w, self.b])
        train_fn = th.function(
            inputs=[self.X, self.y],
            outputs=hinge_loss,
            updates=[(self.w, self.w - lr * gw), (self.b, self.b - lr * gb)]
        )
        for epoch in range(epochs):
            train_loss = train_fn(X_train, y_train)
            print(f'Theano: Epoch [{epoch+1}/{epochs}], Loss: {train_loss:.4f}')

In [ ]:
# Instantiate and train the model
svm_theano = SVMTheano()
svm_theano.train(X_train, y_train)

In [ ]:
eng = matlab.engine.start_matlab()
eng.addpath('/path/to/your/MATLAB/script')  # Add the path to your MATLAB script
eng.train_svm(X_train.tolist(), y_train.tolist(), nargout=0)  # Assuming your MATLAB script has a train_svm function

eng.quit()